In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import decomposition

In [28]:
# retrieving data from file. 
x_train = np.loadtxt("x_train.txt") 
  
# This loadedArr is a 2D array, therefore 
# we need to convert it to the original 
# array shape.reshaping to get original 
# matrice with original shape. 
x_train = x_train.reshape( 
    x_train.shape[0], x_train.shape[1] // 73, 73) 

In [31]:
pca = decomposition.PCA(10)
x_train_pca = np.empty((x_train.shape[0],100,10))

for idx, i in enumerate(x_train):
    
    x_train_pca[idx] = pca.fit_transform(i[:,5:])

In [35]:
# reshaping the array from 3D 
# matrice to 2D matrice. 
x_train_reshaped = x_train_pca.reshape(x_train_pca.shape[0], -1) 
  
# saving reshaped array to file. 
np.savetxt("x_train_pca.txt", x_train_reshaped) 

In [19]:
import numpy as np
import os
import shutil

In [27]:
### move x% of files to validation folder

val_pct = 0.2

for i in ["buy/", "sell/"]:
    files = os.listdir("train/" + i)
    
    # get index of only candles where signal occurs, a randome sample that is equal to number of buy/sell signals divided by 2 to keep a balance in trianing set:
    np.random.seed(42)
    files_to_move = np.random.choice(files, replace=False, size=round(len(files)*val_pct))

    #files_to_move = files[:round(len(files)*val_pct)]
    
    if not os.path.exists("validation/"+i):
        os.makedirs("validation/"+i)
    for f in files_to_move:
        shutil.move("train/" + i + f,"validation/" + i) #change move to copy if you want to copy insted of moving 

In [31]:
ad = np.genfromtxt('../financial_data/eurusd.csv', delimiter=',' ,dtype=str)
pd = np.flipud(ad)

In [41]:
[print(x) for x in range(12-0)]

0
1
2
3
4
5
6
7
8
9
10
11


[None, None, None, None, None, None, None, None, None, None, None, None]

In [26]:
from numpy import genfromtxt
import matplotlib.pyplot as plt
import mpl_finance
import numpy as np
import uuid

# Input your csv file here with historical data

ad = genfromtxt('../financial_data/eurusd.csv', delimiter=',' ,dtype=str)
pd = np.flipud(ad)

buy_dir = '../data/train/buy/'
sell_dir = '../data/train/sell/'

def convolve_sma(array, period):
    return np.convolve(array, np.ones((period,))/period, mode='valid')

def graphwerk(start, finish):
    open = []
    high = []
    low = []
    close = []
    volume = []
    date = []
    for x in range(finish-start):

# Below filtering is valid for eurusd.csv file. Other financial data files have different orders so you need to find out
# what means open, high and close in their respective order.

        open.append(float(pd[start][1]))
        high.append(float(pd[start][2]))
        low.append(float(pd[start][3]))
        close.append(float(pd[start][4]))
        volume.append(float(pd[start][5]))
        date.append(pd[start][0])
        start = start + 1

    close_next = float(pd[finish][4])

    sma = convolve_sma(close, 5)
    smb = list(sma)
    diff = sma[-1] - sma[-2]

    for x in range(len(close)-len(smb)):
        smb.append(smb[-1]+diff)

    fig = plt.figure(num=1, figsize=(3, 3), dpi=50, facecolor='w', edgecolor='k')
    dx = fig.add_subplot(111)
    #mpl_finance.volume_overlay(ax, open, close, volume, width=0.4, colorup='b', colordown='b', alpha=1)
    mpl_finance.candlestick2_ochl(dx,open, close, high, low, width=1.5, colorup='g', colordown='r', alpha=0.5)

    plt.autoscale()
    plt.plot(smb, color="blue", linewidth=10, alpha=0.5)
    plt.axis('off')
    comp_ratio = close_next / close[-1]
    print(comp_ratio)

    if close[-1] > close_next:
            print('close value is bigger')
            print('last value: ' + str(close[-1]))
            print('next value: ' + str(close_next))
            print('sell')
            plt.savefig(sell_dir + str(uuid.uuid4()) +'.jpg', bbox_inches='tight')
    else:
            print('close value is smaller')
            print('last value: '+ str(close[-1]))
            print('next value: ' + str(close_next))
            print('buy')
            plt.savefig(buy_dir + str(uuid.uuid4())+'.jpg', bbox_inches='tight')


    #plt.show()
    open.clear()
    close.clear()
    volume.clear()
    high.clear()
    low.clear()
    plt.cla()
    plt.clf()



iter_count = int(len(pd)/4)
print(iter_count)
iter = 0


for x in range(len(pd)-4):
   graphwerk(iter, iter+12)
   iter = iter + 2

train/      validation/
